In [1]:
from flask import Flask , request , jsonify
import pyodbc
import pymongo
from pymongo import MongoClient
from bson.objectid import ObjectId
import threading
import time
from datetime import datetime
import requests
import json
import pandas as pd

In [2]:
app = Flask(__name__)


In [ ]:
# SQL Server connection
def connect_db():
    return pyodbc.connect(
        "DRIVER={SQL Server};"
        'SERVER=localhost\\SQLEXPRESS;'
        'DATABASE=RetailSales;'     
        "Trusted_Connection=yes;"
    )

In [ ]:
# Helper function: bulk insert
def bulk_insert(df, table_name, cursor):
    cols = ",".join(df.columns)
    placeholders = ",".join(["?"] * len(df.columns))
    query = f"INSERT INTO {table_name} ({cols}) VALUES ({placeholders})"
    for _, row in df.iterrows():
        cursor.execute(query, tuple(row))

# Load Customers
@app.route("/load/customers", methods=["POST"])
def load_customers():
    try:
        df = pd.read_csv("Customers.csv")
        conn = connect_db()
        cursor = conn.cursor()
        bulk_insert(df, "Customers", cursor)
        conn.commit()
        cursor.close()
        conn.close()
        return jsonify({"message": f"{len(df)} customers inserted successfully"})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Load Branches
@app.route("/load/branches", methods=["POST"])
def load_branches():
    try:
        df = pd.read_csv("Branches.csv")
        conn = connect_db()
        cursor = conn.cursor()
        bulk_insert(df, "Branches", cursor)
        conn.commit()
        cursor.close()
        conn.close()
        return jsonify({"message": f"{len(df)} branches inserted successfully"})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Load Products
@app.route("/load/products", methods=["POST"])
def load_products():
    try:
        df = pd.read_csv("Products.csv")
        conn = connect_db()
        cursor = conn.cursor()

        for _, row in df.iterrows():
            cursor.execute("""
                MERGE Products AS target
                USING (SELECT ? AS ProductID, ? AS ProductName, ? AS Category, ? AS Brand, ? AS UnitPrice) AS source
                ON target.ProductID = source.ProductID
                WHEN MATCHED THEN
                    UPDATE SET 
                        ProductName = source.ProductName,
                        Category = source.Category,
                        Brand = source.Brand,
                        UnitPrice = source.UnitPrice
                WHEN NOT MATCHED THEN
                    INSERT (ProductID, ProductName, Category, Brand, UnitPrice)
                    VALUES (source.ProductID, source.ProductName, source.Category, source.Brand, source.UnitPrice);
            """, (
                row["ProductID"],
                row["ProductName"],
                row["Category"],
                row["Brand"],
                row["UnitPrice"]
            ))

        conn.commit()
        cursor.close()
        conn.close()
        return jsonify({"message": f"{len(df)} products upserted successfully"})

    except Exception as e:
        return jsonify({"error": str(e)}), 500


#  Load Sales
@app.route("/load/sales", methods=["POST"])
def load_sales():
    try:
        df = pd.read_csv("Sales.csv")
        conn = connect_db()
        cursor = conn.cursor()
        bulk_insert(df, "Sales", cursor)
        conn.commit()
        cursor.close()
        conn.close()
        return jsonify({"message": f"{len(df)} sales inserted successfully"})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

#  Load SaleDetails
@app.route("/load/sale_details", methods=["POST"])
def load_sale_details():
    try:
        df = pd.read_csv("SaleDetails.csv")
        conn = connect_db()
        cursor = conn.cursor()
        bulk_insert(df, "SaleDetails", cursor)
        conn.commit()
        cursor.close()
        conn.close()
        return jsonify({"message": f"{len(df)} sale details inserted successfully"})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

#Load Reviews
@app.route("/load/reviews", methods=["POST"])
def load_reviews():
    try:
        df = pd.read_csv("Reviews.csv")
        conn = connect_db()
        cursor = conn.cursor()
        bulk_insert(df, "Reviews", cursor)
        conn.commit()
        cursor.close()
        conn.close()
        return jsonify({"message": f"{len(df)} reviews inserted successfully"})
    except Exception as e:
        return jsonify({"error": str(e)}), 500


    
def run_flask():
    app.run(port=5000 , debug=False , use_reloader=False)

# code to run the flask API in a separate thread in the background
threading.Thread(target=run_flask).start()
time.sleep(1)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


127.0.0.1 - - [27/Sep/2025 19:19:51] "POST /load/products HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2025 19:19:51] "POST /load/products HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2025 19:19:51] "POST /load/products HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2025 19:19:51] "POST /load/products HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2025 19:19:51] "POST /load/products HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2025 19:19:52] "POST /load/products HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2025 19:19:52] "POST /load/products HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2025 19:19:52] "POST /load/products HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2025 19:19:52] "POST /load/products HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2025 19:19:53] "POST /load/products HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2025 19:19:53] "POST /load/products HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2025 19:19:53] "POST /load/products HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2025 19:19:53] "POST /load/products HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2025 19:19:53] "

In [ ]:
# Endpoints for each table
endpoints = {
    "Customers.csv": "http://127.0.0.1:5000/load/customers",
    "Branches.csv": "http://127.0.0.1:5000/load/branches",
    "Products.csv": "http://127.0.0.1:5000/load/products",
    "Sales.csv": "http://127.0.0.1:5000/load/sales",
    "SaleDetails.csv": "http://127.0.0.1:5000/load/sale_details",
    "Reviews.csv": "http://127.0.0.1:5000/load/reviews"
}

# Send each CSV row to its endpoint
for file_name, url in endpoints.items():
    try:
        df = pd.read_csv(file_name)

        for _, row in df.iterrows():
            record = row.to_dict()
            res = requests.post(url, json=record)
            print(f"POST to {url} -> {res.status_code} {res.text}")

    except FileNotFoundError:
        print(f"❌ File {file_name} not found!")
    except Exception as e:
        print(f"❌ Error with {file_name}: {str(e)}")


POST to http://127.0.0.1:5000/load/products -> 200 {"message":"500 products upserted successfully"}

POST to http://127.0.0.1:5000/load/products -> 200 {"message":"500 products upserted successfully"}

POST to http://127.0.0.1:5000/load/products -> 200 {"message":"500 products upserted successfully"}

POST to http://127.0.0.1:5000/load/products -> 200 {"message":"500 products upserted successfully"}

POST to http://127.0.0.1:5000/load/products -> 200 {"message":"500 products upserted successfully"}

POST to http://127.0.0.1:5000/load/products -> 200 {"message":"500 products upserted successfully"}

POST to http://127.0.0.1:5000/load/products -> 200 {"message":"500 products upserted successfully"}

POST to http://127.0.0.1:5000/load/products -> 200 {"message":"500 products upserted successfully"}

POST to http://127.0.0.1:5000/load/products -> 200 {"message":"500 products upserted successfully"}

POST to http://127.0.0.1:5000/load/products -> 200 {"message":"500 products upserted succes